In [1]:
import os

# --- Add these lines at the very top of your script ---
# This must be done BEFORE importing numpy or other scientific libraries.
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
import time
from device import Device
from device import Device
from rgf import GreensFunction
import scipy as sp
from hamiltonian import Hamiltonian
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp
from lead_self_energy import LeadSelfEnergy
from scipy.sparse import bmat, identity, random, csc_matrix
from scipy.sparse.linalg import eigsh, eigs, spsolve
import time
from NEGF_unit_generation import UnitCell


In [4]:
# Test code to isolate dangling bond effects
dev = Device(2.5e-9, 1.5e-9)
ham = Hamiltonian(dev)

# Create unit cell with dangling bonds
unit_cell = UnitCell(dev.unitZ, dev.unitX, orientiation=(0, 1, 2, 3), not_NEGF=True)

# Create Hamiltonian WITHOUT potential correction
H = ham.create_sparse_channel_hamlitonian(ky=0, blocks=False, unitCell=unit_cell)

# Check eigenvalues
eigvals, eigvecs = spa.linalg.eigsh(H, k=max(50, H.shape[0]-2), which='LM')
eigvals = np.sort(eigvals.real)

# Look for bandgap
print("Eigenvalues around Fermi level:")
for i, val in enumerate(eigvals):
    if -5 < val < 5:  # Look around Fermi level
        print(f"E{i}: {val:.3f} eV")

Eigenvalues around Fermi level:
E77: -4.794 eV
E78: -4.681 eV
E79: -4.561 eV
E80: -4.554 eV
E81: -4.545 eV
E82: -4.506 eV
E83: -4.483 eV
E84: -4.409 eV
E85: -4.404 eV
E86: -4.333 eV
E87: -4.315 eV
E88: -4.300 eV
E89: -4.234 eV
E90: -4.083 eV
E91: -4.063 eV
E92: -4.036 eV
E93: -4.008 eV
E94: -3.998 eV
E95: -3.983 eV
E96: -3.949 eV
E97: -3.914 eV
E98: -3.897 eV
E99: -3.815 eV
E100: -3.788 eV
E101: -3.778 eV
E102: -3.763 eV
E103: -3.739 eV
E104: -3.720 eV
E105: -3.695 eV
E106: -3.675 eV
E107: -3.563 eV
E108: -3.553 eV
E109: -3.523 eV
E110: -3.506 eV
E111: -3.444 eV
E112: -3.441 eV
E113: -3.414 eV
E114: -3.390 eV
E115: -3.346 eV
E116: -3.331 eV
E117: -3.313 eV
E118: -3.306 eV
E119: -3.293 eV
E120: -3.286 eV
E121: -3.255 eV
E122: -3.247 eV
E123: -3.229 eV
E124: -3.212 eV
E125: -3.183 eV
E126: -3.147 eV
E127: -3.137 eV
E128: -3.067 eV
E129: -3.006 eV
E130: -2.995 eV
E131: -2.986 eV
E132: -2.959 eV
E133: -2.911 eV
E134: -2.886 eV
E135: -2.843 eV
E136: -2.830 eV
E137: -2.787 eV
E138: -2.768 eV

In [3]:
for i in range(eigvals.shape[0]):
    print(eigvals[i])

25.88054376539796
25.92365237804689
25.978384660927915
26.063892372157554
26.132022418112143
26.470330058608948
26.82167483124887
26.905594166406768
26.96566722794447
27.32275049352692
27.54734443307286
27.689565251713613
27.99227144829518
28.235556199144767
28.399347021398388
28.936364615718038
29.550037400192277
29.705469122991378
29.829487973458527
30.27332520493653
30.276330223625326
30.622084462439307
31.06861373920679
31.242611122937593
32.07677620119004
32.13209643324461
32.70870802614832
32.82097695704383
33.277102955313296
33.74102111014277
33.90134442543237
34.78174800049347
35.05030325558374
35.32160629610289
36.01717802549126
36.610202044990785
158.10437403655382
158.1056981814467
158.10783758226424
158.11068846754537
158.11409634502067
158.11791060017939
158.12192783728287
158.1259534404447
158.12978731700954
158.13323071347438
158.1361131146767
158.1382846970463
158.13963451905084
237.93749547201722


In [ ]:

def DOS(param): 
    
    E, ky = param
    dev = Device(5e-9, 1e-9)
    
    ham = Hamiltonian(dev)


    lse = LeadSelfEnergy(dev, ham)

    H = ham.create_sparse_channel_hamlitonian(ky, blocks=False)

    sl = csc_matrix(lse.self_energy(side="left",E =E,ky =  ky))
    sr = csc_matrix(lse.self_energy(side="right",E =E,ky =  ky))
    block_size = sl.shape[0]
    H[:block_size, :block_size] += sl
    H[-block_size:,-block_size:] += sr

    H = csc_matrix(np.eye(H.shape[0], dtype=complex) * E) - H

    I = csc_matrix(np.eye(H.shape[0], dtype=complex))

    H.shape[0]

    A = spsolve(H,I)
    tr = -1/np.pi * np.imag(A.trace())

    return (E, ky, tr)


energy_values = np.linspace(-2, 3, 50)  # More energy points for better resolution
ky_values = np.linspace(0, 1.0, 20)     # Fewer ky points for faster computation
dk = ky_values[1] - ky_values[0]        # The differential k_y step

param_grid = list(product(energy_values, ky_values))

print(f"Starting DOS calculations for {len(param_grid)} (E, ky) pairs...")
print(f"Energy range: {energy_values[0]:.2f} to {energy_values[-1]:.2f} eV")
print(f"ky range: {ky_values[0]:.2f} to {ky_values[-1]:.2f}")

start_time = time.time()

# Use fewer processes for debugging - increase to 32 once working
with multiprocessing.Pool(processes=8) as pool:
    results = pool.map(DOS, param_grid)
    
end_time = time.time()
print(f"Calculations finished in {end_time - start_time:.2f} seconds.")

# Aggregate traces for each energy value (sum over all ky values)
energy_traces = defaultdict(complex)
valid_results = 0

for energy, ky, trace in results:
    if not (np.isnan(trace) or np.isinf(trace)):
        energy_traces[energy] += trace
        valid_results += 1
    else:
        print(f"Invalid result at E={energy:.3f}, ky={ky:.3f}: {trace}")

print(f"Valid results: {valid_results}/{len(results)}")

# Sort energies and calculate DOS
dos_energies = sorted(energy_traces.keys())

# DOS formula: D(E) = -(1/π) * Im[Tr(G_R(E))] * (dk/2π)

dos_values = []
for E in dos_energies:
    trace_val = energy_traces[E]
    dos_val = trace_val *  (dk / (2 * np.pi))
    dos_values.append(dos_val)

print(f"DOS calculation complete. Energy points: {len(dos_energies)}")
# Plotting the Density of States
plt.figure(figsize=(12, 8))

# Main DOS plot
plt.subplot(2, 1, 1)
plt.plot(dos_energies, dos_values, 'b-', linewidth=2, label='Density of States D(E)')
plt.xlabel('Energy (eV)')
plt.ylabel('DOS (states/eV)')
plt.title('Density of States from Green\'s Function')
plt.grid(True, alpha=0.3)
plt.legend()



plt.tight_layout()
plt.savefig("dos.png", dpi=300, bbox_inches='tight')
plt.show()

# Save data for further analysis
np.savetxt("dos_data.txt", np.column_stack([dos_energies, dos_values]), 
            header="Energy(eV)\tDOS(states/eV)", fmt="%.6e")

In [ ]:
DOS((0,0))

/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)


(0, 0, np.float64(48.5218780992764))

In [ ]:
from poisson import PoissonSolver

poissonSolver = PoissonSolver(device)

In [ ]:
poissonSolver.solve_poisson_equation()



In [ ]:
device = Device()
ham = Hamiltonian(device)
rgf = GreensFunction(device, ham)

G_R, gamma1, gamma2, sigmaL, sigmaR= rgf.sparse_rgf_G_R(E=0.1, ky = 0.1)


In [ ]:
print(G_R[0].shape)

In [ ]:
time_start = time.time()
dagger = lambda A: np.conjugate(A.T)
side = "left"
Hpp_matrices = [None] * lead.P
HpP_matrices = [None] * lead.P
hPP,hPP1 = lead.get_layer_hamiltonian(lead.P, side)

HPP = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - hPP))
Hpp_matrices[-1], HpP_matrices[-1] = HPP, HPP
for i in range(lead.P - 1, 0, -1):
    
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    Hpp = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - \
        hpp - hpp1 @ Hpp_matrices[i] @ dagger(hpp1)))
    Hpp_matrices[i - 1] = Hpp
    HpP = Hpp_matrices[i - 1] @ hpp1 @ HpP_matrices[i]
    HpP_matrices[i - 1] = HpP
    
C22 = Hpp_matrices[1]
C2P = HpP_matrices[1]
C_matrices = [None] * 4
C_matrices[1] = C22
for p in range(3, lead.P + 1):
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    C_matrices[p - 1] = Hpp_matrices[p - 1] + Hpp_matrices[p -1] @ (hpp1 @ C_matrices[p -2] @ dagger(hpp1)) @ Hpp_matrices[p - 1] 

h11, h12 = lead.get_layer_hamiltonian(1, side)

XIs = h11 + h12 @ C_matrices[1] @ dagger(h12)
XI = XIs + dagger(hPP1) @ C_matrices[-1] @ hPP1
PI = h12 @ C2P @ hPP1





In [ ]:
XIs = spa.csc_matrix(XIs)
XI = spa.csc_matrix(XI)
PI = spa.csc_matrix(PI)
I = np.eye(XI.shape[0], dtype=XI)
Z = I * 0
D = lead.E * I.copy() - XI
T = -PI

A = bmat([
    [Z, I],
    [-T.conj().T, -D]
], format='csc')

B = bmat([
    [I, Z],
    [Z, T]
], format='csc')

eigenvalues, eigenvectors = eigs(A, M=B, sigma=1.0, which='LM')




def construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, n_dim, epsilon=0.1):
    abs_vals = np.abs(eigenvalues)
    

    is_propagating = np.isclose(abs_vals, 1.0)
    is_evanescent = (abs_vals < 1.0) & (abs_vals > epsilon)
    
    selected_indices = np.where(is_propagating | is_evanescent)[0]
    
    if len(selected_indices) == 0:
        return np.array([], dtype=complex), np.array([],dtype=complex)
        
    filtered_eigenvalues = eigenvalues[selected_indices]
    filtered_eigenvectors = eigenvectors[:, selected_indices]

    Lambda_plus = np.diag(filtered_eigenvalues)
    U_plus = filtered_eigenvectors[:n_dim, :]

    return U_plus, Lambda_plus

U_plus, Lambda = construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, T.shape[0], epsilon=0.1)

U_pseudo = np.linalg.pinv(U_plus)
F = U_plus @ Lambda @ U_pseudo

Y = np.linalg.solve(lead.E * I - XIs.toarray() - PI.toarray() @ F, dagger(h12.toarray()))
self_energy = h12 @ Y
time_end = time.time()
print(f"time is: {time_end - time_start}")

In [ ]:
print(self_energy)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)

In [ ]:
def surface_gf(Energy, H00, H10, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """

    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0
    first_time = True

    while err > tol:
        if first_time:
            inv_E = Helper_functions.sparse_inverse(spa.csr_matrix(Energy * I) - spa.csr_matrix(epsilon))
            first_time = False
        else:

            inv_E = np.linalg.solve(Energy * I - epsilon, I)
    
        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)


surface_gf(0, H00, H10, tol=1e-3)

In [ ]:
def surface_gf(Energy, H00 : np.ndarray, H10: np.ndarray, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """
    
    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0

    while err > tol:
        inv_E = np.linalg.solve(Energy * I - epsilon, I)

        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)

surface_gf(0, H00, H10)

In [ ]:
from helper import Helper_functions

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)
Helper_functions.sparse_inverse(-H00)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)
Helper_functions.sparse_inverse(-H00)

In [ ]:
H = device.hamiltonian.create_sparse_hamlitonian(0.1)